<a href="https://colab.research.google.com/github/filiperobotic/cursoDL/blob/master/codes/aula7/fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/filiperobotic/aula7.git


In [0]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Model
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.merge import concatenate
from keras.layers import Input
from keras import backend as K

In [0]:
def build_single(width, height, depth, classes):

    inputShape = (height, width, depth)
    
    input1 = Input(shape=inputShape)
    x = Conv2D(64, (11,11), input_shape=inputShape,
			padding="same")(input1)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, (5, 5), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(256, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(64)(x)
    x = Activation("relu")(x)

    x = Dense(classes)(x)
    x = Activation("softmax")(x)
    
    model = Model(inputs=input1, outputs=x)
    
    # return the constructed network architecture
    return model

In [0]:
def build_double(width, height, depth, classes):
		
    #TODO
    
    # return the constructed network architecture
    return model

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
#from pyimagesearch.simplenet import SimpleNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from imutils import build_montages
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

arg_dataset1 = "aula7/lung/planox"
arg_dataset2 = "aula7/lung/planoy"
arg_epochs = 30

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths1 = list(paths.list_images(arg_dataset1))
imagePaths2 = list(paths.list_images(arg_dataset2))
data1 = []
labels1 = []

# loop over the image paths 1
for imagePath in imagePaths1:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the image, convert it to grayscale, and resize it to be a
	# fixed 64x64 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.resize(image, (64, 64))

	# update the data and labels lists, respectively
	data1.append(image)
	labels1.append(label)
  
data2 = []
#labels2 = labels1
  
# loop over the image paths 2
for imagePath in imagePaths2:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the image, convert it to grayscale, and resize it to be a
	# fixed 64x64 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.resize(image, (64, 64))

	# update the data and labels lists, respectively
	data2.append(image)

# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data1 = np.array(data1, dtype="float") / 255.0
data2 = np.array(data2, dtype="float") / 255.0

# reshape the data matrix so that it explicity includes a channel
# dimension
data1 = data1.reshape((data1.shape[0], data1.shape[1], data1.shape[2], 1))
data2 = data2.reshape((data2.shape[0], data2.shape[1], data2.shape[2], 1))

# encode the labels (which are currently strings) as integers
le = LabelEncoder()
labels = le.fit_transform(labels1)

# transform the labels into vectors in the range [0, classes],
# generating a vector for each label, where the index of the label
# is set to '1' and all other entries are set to '0' -- this process
# is called "one-hot encoding"
labels = np_utils.to_categorical(labels, 2)

(trainX_d1, testX_d1, trainY_d1, testY_d1) = train_test_split(data1, labels,
 	test_size=0.40, stratify=labels, random_state=42)

(trainX_d2, testX_d2, trainY_d2, testY_d2) = train_test_split(data2, labels,
 	test_size=0.40, stratify=labels, random_state=42)

# initialize the optimizer and model
print("[INFO] compiling model...")

opt = Adam(lr=1e-5, decay=1e-5 / arg_epochs)
model = build_single(width=64, height=64, depth=1,
	classes=2)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(
	arg_epochs))
H = model.fit(trainX_d1, trainY, validation_data=(testX_d1, testY_d1),
	batch_size=32, epochs=arg_epochs, verbose=1)



In [0]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX_d1, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))